In [1]:
import pandas as pd
import re
from nltk import ngrams
from underthesea import word_tokenize
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib
import os

In [2]:
df = pd.read_csv("data1.csv").drop([0])
train = df["Title"]
df.head(7)

,Title,target
1,"Đã nhận được hàng, máy đẹp, giao hàng nhanh. C...",0
2,Tôi mua sản phẩm samsung Galaxy M10. Máy bị lỗ...,1
3,"Đặt Samsung Galaxy M10,giao hàng ngoài hộp M10...",1
4,"Hàng đẹp xài ngon so với giá tiền, nên mua cho...",0
5,Lúc mua cái màu đen nó nóng quá vs mau hết pin...,1
6,"shop rất uy tín, ban đầu lỗi xíu, yêu cầu đổi ...",0
7,ok thiếu củ sạc,1


In [3]:
def transform_row(row):
    #remove stopword
    
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
    
    row = row.strip()
    return row 

df["Title"] = df.Title.apply(transform_row)
df.head(7)

,Title,target
1,Đã nhận được hàng máy đẹp giao hàng nhanh C...,0
2,Tôi mua sản phẩm samsung Galaxy M10 Máy bị lỗ...,1
3,Đặt Samsung Galaxy M10 giao hàng ngoài hộp M10...,1
4,Hàng đẹp xài ngon so với giá tiền nên mua cho...,0
5,Lúc mua cái màu đen nó nóng quá vs mau hết pin...,1
6,shop rất uy tín ban đầu lỗi xíu yêu cầu đổi ...,0
7,ok thiếu củ sạc,1


In [4]:
from sklearn.model_selection import train_test_split
def data_ngram(row):
    row = word_tokenize(row, format="text")
    return row
df["Title"] = df.Title.apply(data_ngram)
x_train,x_test,y_train,y_test = train_test_split(df["Title"],df["target"],test_size=0.3, random_state=42)

In [5]:
x_train

31     Mặc_dù túi và hình_ảnh trên ly k liên_quan như...
78     Sản_phẩm tốt Nhựa cứng lò_xo chắc_chắn Bị cái ...
66     hàng đẹp nhưng chát ko được chỉ nghe chứ nói b...
10                          hag chuan rat ug y cam on sh
34     shop nhiệt_tình ghê luôn nhắn_tin hỏi mình chọ...
26                  san pham kem chat luong that te luon
71                  đặt 3 lần 1 màu về 3 màu shop quá tệ
29                        đẹp chat luong cung ổn nên mua
121             dkm shop bán hàng cũ bào hàng mới bố_láo
112                                  balo bị đứt dây nha
6      shop rất uy_tín ban_đầu lỗi xíu yêu_cầu đổi hà...
65     mình mới mua xong sp tốt vượt mong_đợi các bạn...
70              Đặt full đỏ nhưng ra mấy cái màu tài lao
40     đẹp đến từng cm cảm_ơn xẽ ủng_hộ lâu_dài nêu c...
50                           sao ko mo len duoc vay shop
36     rất hài_lòng chỉ điều là hình màu trắng nhưng ...
17     quân lot su dp giôg mâu nhưg hang ko tôt như j...
68                             

In [6]:
train = TfidfVectorizer(min_df=5, max_df=0.8,max_features=3000,sublinear_tf=True)
X = train.fit(x_train)
joblib.dump(train, 'tfidf.pkl')

['tfidf.pkl']

In [ ]:
X = train.transform(x_train)
X
Y = y_train

In [ ]:
X.shape

In [ ]:
if os.path.isfile('saved_model.pkl') == False:
    model = svm.SVC(kernel='linear', C=1)
    model.fit(X,Y)
    joblib.dump(model, 'saved_model.pkl') 
else:
    model = joblib.load('saved_model.pkl')

In [ ]:
X_test = train.transform(x_test)
X_test = X_test.toarray()
X_test.shape

In [ ]:
x_test

In [ ]:
Y_Test = y_test
model.score(X_test,Y_Test)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameter_candidates = [
  {'C': [0.001, 0.01, 0.1, 1, 5, 10, 100, 1000], 'kernel': ['linear','rbf']},
]

clf = GridSearchCV(estimator=svm.SVC(), param_grid=parameter_candidates, n_jobs=-1)
clf.fit(X, Y)
print('Best score:', clf.best_score_)
print('Best C:',clf.best_estimator_.C)
print('Best kernel:',clf.best_estimator_.kernel)